In [2]:
from functools import reduce
import gc
import numpy as np

"""STVM 계산(3차로 점선) + 지상부 추가"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 5400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 본선부 검지기
main_line = 60

# 지상부_진출 검지기
exit_line = 265
############################


######## 램프 ###############
# 유입램프
input_ramp = 59

# 유출램프
output_ramp = 61
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(df):
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도
def density_mean(df):
    density_mean_df = df.assign(K = df["V_count"] * 12 / df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):

    # 진입부 검지기 선정
    measurement = enter_line

    heavy_df = (
        original_df[original_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        original_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 진입 포화도
def entry_saturation(original_df):
    # 실측용량 C
    max_capacity = 2200
    entry_saturation_df = (
        # 진입부에서 유입된 교통량이므로 진입부 중 한 개의 검지기를 선정하여 측정
        original_df[original_df["New_Measurement"] == enter_line]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df


# 램프 유출입 비율
def rfr_rate(original_df):
    """
    1. 본선 1개의 검지기 data 수집(New_Measurement = 60)
    2. 위의 검지기 앞, 뒤 검지기 data (59 / 61)
    3. RFR 연산
    """
    main_df = (
        original_df[original_df["New_Measurement"] == main_line] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == input_ramp] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == output_ramp] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0.2값으로 fix
    main_df["RFR"] = 0.2
    return main_df

# 유출 정상성 비율
def output_normality(original_df):
    # 6번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    entry_df = original_df[original_df["New_Measurement"] == enter_line][["VehNo", "t(Entry)"]]

    # 265번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    exit_df = original_df[original_df["New_Measurement"] == exit_line][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )
    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"] == enter_line]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"] == exit_line]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")

    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

    # 1−F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)
    merged_counts["1-F(outrate)"] = 1 - merged_counts["F(outrate)"]
    return merged_counts


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):
    # TimeGroup 기준으로  Merge
    merged = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
    .merge(entry_saturation_df[["TimeGroup", "Phi_진입"]], on="TimeGroup")
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "1-F(outrate)"]], on="TimeGroup")
    )


    merged["STVM"] = (
        weights["w1"] * merged["delta_V"] +
        weights["w2"] * merged["delta_K"] +
        weights["w3"] * merged["rate"] +
        weights["w4"] * merged["Phi_진입"] +
        weights["w5"] * merged["RFR"] +
        weights["w6"] * merged["1-F(outrate)"]
    )
    """
    merged_df = merged.pivot(index='TimeGroup',
                columns='New_Measurement',
                values='STVM')
    merged_df = merged_df.sort_index().sort_index(axis=1)
    modify_stvm_df = modify_frame(merged_df)
    """
    return merged


def calculate_z_score(stvm_df):
    # 평균
    mean_stvm = stvm_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = stvm_df["STVM"].std(skipna=True)

    # Z-Score 계산
    stvm_df["Z-Score"] = (stvm_df["STVM"] - mean_stvm) / std_stvm
    z_max = stvm_df["Z-Score"].max()
    z_min = stvm_df["Z-Score"].min()

    stvm_df["환산점수"] = stvm_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(stvm_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")

    return stvm_df

def modify_frame(z_score_df):
    result_df = z_score_df.copy()
    result_df = result_df.drop(columns=list(range(1,6)) + list(range(276,281)))

    columns = result_df.columns.tolist()

    new_columns = [-5, -4, -3, -2, -1]
    # 나머지 (11번 이후) → 1부터 순차 번호
    new_columns += list(range(1, len(columns) - 5 + 1))
    result_df.columns = new_columns
    return result_df

def calculate_avg(df):

    def avg_range(df, low, high):
        cols = [c for c in df.columns if low <= c <= high]
        return df[cols].mean(axis=1) if cols else np.nan

    avg_df = pd.DataFrame(index=df.index)
    avg_df["지상부_진입"] = avg_range(df, -5, -1)
    avg_df["진입부"] = avg_range(df, 1, 30)
    avg_df["본선부"] = avg_range(df, 31, 230)
    avg_df["진출부"] = avg_range(df, 231, 260)
    avg_df["지상부_진출"] = avg_range(df, 261, 265)

    return avg_df

def merged_varible(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):
    # New_Measurement 별 피봇
    pivot_speed_df = speed_df.pivot(index='TimeGroup',
                columns='New_Measurement',
                values='delta_V')
    pivot_speed_df = pivot_speed_df.sort_index().sort_index(axis=1)

    pivot_density_df = density_df.pivot(index='TimeGroup',
            columns='New_Measurement',
            values='delta_K')
    pivot_density_df = pivot_density_df.sort_index().sort_index(axis=1)

    speed_new_df = modify_frame(pivot_speed_df)
    density_new_df = modify_frame(pivot_density_df)

    avg_speed_df = calculate_avg(speed_new_df)
    avg_density_df = calculate_avg(density_new_df)

    merged_df_list = [avg_speed_df, avg_density_df, heavy_df[["TimeGroup", "rate"]], entry_saturation_df[["TimeGroup", "Phi_진입"]], rfr_df[["TimeGroup", "RFR"]], normality_df[["TimeGroup", "1-F(outrate)"]]]

    merged_df = reduce(lambda left, right: pd.merge(left, right, on="TimeGroup", how="left"), merged_df_list)

    return merged_df


def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")


def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\mer파일\1350"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()

for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 평균속도
                speed_df = speed_mean(original_df)
                display("speed_df : ", speed_df)
                # 밀도
                density_df = density_mean(speed_df)
                display("density_df : ", density_df)

                # 가중 중차량 비율
                heavy_df = heavy_rate(original_df)
                display("heavy_df : ", heavy_df)
                # 진입 포화도
                entry_saturation_df = entry_saturation(original_df)
                display("entry_saturation_df : ", entry_saturation_df)

                # 램프 유출입 비율
                rfr_df = rfr_rate(original_df)
                display("rfr_df : ", rfr_df)

                # 진출 정상성
                normality_df = output_normality(original_df)
                display("normality_df : ", normality_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #save_to_excel(stvm_df, folder_path, "STVM", i)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)

                # 결과 정리
                result_df = modify_frame(z_score_df)

                # 변수별 데이터 모음(STVM 결과랑 상관 X)
                #varible_df = merged_varible(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)

                #save_to_excel(result_df, folder_path, "환산점수", i)
                #save_to_excel(varible_df, folder_path, "세부항목", i)

                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()

'speed_df : '

,TimeGroup,New_Measurement,V_mean,V_count,V_next,delta_V
0,1800~2100,1,104.857925,347,105.072754,0.002049
1,1800~2100,2,105.072754,345,105.063188,-0.000091
2,1800~2100,3,105.063188,345,104.637427,-0.004052
3,1800~2100,4,104.637427,342,104.037609,-0.005732
4,1800~2100,5,104.037609,343,103.579651,-0.004402
...,...,...,...,...,...,...
3355,5100~5400,276,101.557434,343,100.848997,-0.006976
3356,5100~5400,277,100.848997,349,101.307471,0.004546
3357,5100~5400,278,101.307471,348,101.052616,-0.002516
3358,5100~5400,279,101.052616,344,102.289826,0.012243


'density_df : '

,TimeGroup,New_Measurement,V_mean,V_count,V_next,delta_V,K,K_next,delta_K
0,1800~2100,1,104.857925,347,105.072754,0.002049,39.710875,39.401271,-0.007796
1,1800~2100,2,105.072754,345,105.063188,-0.000091,39.401271,39.404858,0.000091
2,1800~2100,3,105.063188,345,104.637427,-0.004052,39.404858,39.221148,-0.004662
3,1800~2100,4,104.637427,342,104.037609,-0.005732,39.221148,39.562616,0.008706
4,1800~2100,5,104.037609,343,103.579651,-0.004402,39.562616,39.853388,0.007350
...,...,...,...,...,...,...,...,...,...
3355,5100~5400,276,101.557434,343,100.848997,-0.006976,40.528791,41.527433,0.024640
3356,5100~5400,277,100.848997,349,101.307471,0.004546,41.527433,41.221047,-0.007378
3357,5100~5400,278,101.307471,348,101.052616,-0.002516,41.221047,40.850006,-0.009001
3358,5100~5400,279,101.052616,344,102.289826,0.012243,40.850006,40.355920,-0.012095


'heavy_df : '

...

,TimeGroup,New_Measurement,entry_volume,Phi_진입
0,1800~2100,6,321,1.750909
1,2100~2400,6,325,1.772727
2,2400~2700,6,297,1.620000
3,2700~3000,6,327,1.783636
4,3000~3300,6,320,1.745455
5,3300~3600,6,322,1.756364
6,3600~3900,6,310,1.690909
7,3900~4200,6,315,1.718182
8,4200~4500,6,331,1.805455
9,4500~4800,6,306,1.669091


'rfr_df : '

,TimeGroup,New_Measurement,main_line,RFR
0,1800~2100,60,337,0.2
1,2100~2400,60,329,0.2
2,2400~2700,60,294,0.2
3,2700~3000,60,313,0.2
4,3000~3300,60,336,0.2
5,3300~3600,60,320,0.2
6,3600~3900,60,318,0.2
7,3900~4200,60,300,0.2
8,4200~4500,60,321,0.2
9,4500~4800,60,342,0.2


'normality_df : '

,TimeGroup,Q_in,Q_out,Q_out_shift,F(outrate),1-F(outrate)
0,1800~2100,321,280,368.0,1.146417,-0.146417
1,2100~2400,325,315,330.0,1.015385,-0.015385
2,2400~2700,297,297,323.0,1.087542,-0.087542
3,2700~3000,327,368,316.0,0.966361,0.033639
4,3000~3300,320,330,251.0,0.784375,0.215625
5,3300~3600,322,323,368.0,1.142857,-0.142857
6,3600~3900,310,316,347.0,1.119355,-0.119355
7,3900~4200,315,251,283.0,0.898413,0.101587
8,4200~4500,331,368,341.0,1.030211,-0.030211
9,4500~4800,306,347,NaN,0.000000,1.000000
